In [11]:
import requests
import time

def transcribe():
    try:
        prompt = """
        你是一個能夠解析並整理學習編碼的 AI 助手。

使用者會提供多個學習編碼的字串（可能以不同格式或自然語言敘述出現）。

請你解析這些字串並產生符合以下 JSON Schema 的嚴格 JSON 格式。



目標 JSON Schema

{
    "name": "codes_metadata",
    "strict": true,
    "schema": {
        "type": "object",
        "description": "Schema that describes how transcript segment and its corresponding codes are structured.",
        "properties": {
            "codes": {
                "description": "A list of learning content or performance codes associated with this segment.",
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        },
        "required": [
            "codes"
        ],
        "additionalProperties": false
    }
}
任務重點

1. 只輸出一個 JSON 物件

• 頂層物件中僅能有 "codes" 一個屬性。

• "codes" 是一個字串陣列，列出所有解析後的編碼。

• 不要在 JSON 中出現額外屬性或描述。

2. 轉換規則

• 從輸入文字中找出所有與學習編碼（如「Eb-Ⅳ-10」、「Bb-Ⅳ-3」等）相符的字串，若有重複則請視需求去留（可重複或去重都可自行決定）。

• 只要字串可以視為「學習編碼」，就將其加入 "codes" 陣列；若沒有符合學習編碼，則產生空陣列 []。

• 若尚無任何規範可用，也可直接將使用者提供的字串，全都塞入 "codes" 陣列。

3. 最終輸出

• 一律輸出標準 JSON（如 {"codes": ["Ab-Ⅳ-2", "Bc-Ⅳ-1"]}）。

• 不要輸出任何額外解釋文字或 Markdown 標記。

• 確保能透過上述 JSON Schema 驗證。



範例

• 輸入（範例）：

我有一些代碼： Jb-Ⅳ-1, Eb-Ⅳ-10, Eb-Ⅳ-10, Fc-Ⅳ-2
不小心多打了一些空格和逗號


• 輸出（範例）：

{
  "codes": ["Jb-Ⅳ-1", "Eb-Ⅳ-10", "Eb-Ⅳ-10", "Fc-Ⅳ-2"]
}
請依照以上規定，將使用者輸入的多個編碼字串，轉成嚴格的 JSON 物件，並確保符合 JSON Schema。"""
        schema = {
    "name": "codes_metadata",
    "strict": True,
    "schema": {
        "type": "object",
        "description": "Schema that describes how transcript segment and its corresponding codes are structured.",
        "properties": {
            "codes": {
                "description": "A list of learning content or performance codes associated with this segment.",
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        },
        "required": [
            "codes"
        ],
        "additionalProperties": False
    }
}
        messages = [
            {"role": "system", "content": [{"type": "text", "text": prompt}]},
            {
                "role": "user",
                "content": [
                    { "type": "text", "text": """["Eb-Ⅳ-10", "Eb-Ⅳ-11", "Eb-Ⅳ-12"]"""}
                ]
            }
        ]

        # 请将以下 URL 替换为实际的 API 端点
        url = 'https://sujfzchcwdlma5-5000.proxy.runpod.net/v1/chat/completions'

        headers = {
            'Content-Type': 'application/json'
            # 如需身份验证，请在此处添加 'Authorization' 字段
        }

        payload = {
            'model': 'Models/Llama-3.3-70B-Instruct',
            'max_tokens': 1024,
            'temperature': 0.6,
            'messages': messages,
            'guided_json':schema
        }

        response = requests.post(url, headers=headers, json=payload)

        if not response.ok:
            raise Exception(f"HTTP error! status: {response.status_code}")

        data = response.json()

        return data

    except Exception as error:
        print('Error:', error)
        raise error

start_time = time.time()
segments = transcribe()
print('Time:', time.time() - start_time)
print(segments)

Time: 4.040237665176392
{'id': 'chatcmpl-0a1586d09a02410e8b56bf67d3a97219', 'object': 'chat.completion', 'created': 1739023106, 'model': 'Models/Llama-3.3-70B-Instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'reasoning_content': None, 'content': '{"codes": ["Eb-Ⅳ-10", "Eb-Ⅳ-11", "Eb-Ⅳ-12"]}', 'tool_calls': []}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None}], 'usage': {'prompt_tokens': 684, 'total_tokens': 712, 'completion_tokens': 28, 'prompt_tokens_details': None}, 'prompt_logprobs': None}


In [12]:
import requests
import time

def transcribe():
    try:
        prompt = """
        你是一個能夠解析輸入文字的 AI 助手。
        輸入一段「非標準 JSON」或「混雜自然語言」的內容，當中包含了 segments、title、text等資訊。你的任務是：
            1.	理解這些自然語言內容或不完整的 JSON；
            2.	抽取出所需的資訊並將其重組為符合以下 JSON Schema的「嚴格 JSON 格式」。
            3.	完整遵守下述 JSON Schema 的結構與屬性格式，才能保證最終輸出的文件能通過驗證。

        ### JSON Schema Start ###
        {
            "name": "segments_metadata",
            "strict": true,
            "schema": {
                "type": "object",
                "description": "Schema that describes how transcript segments are structured.",
                "properties": {
                    "segments": {
                        "description": "The segmented results from the transcript. Each item is an individual segment.",
                        "type": "array",
                        "items": {
                            "type": "object",
                            "description": "A single segment containing the text.",
                            "properties": {
                                "title": {
                                    "type": "string",
                                    "description": "The actual title abount the segment."
                                },
                                "text": {
                                    "type": "string",
                                    "description": "The actual text contained within this segment."
                                }
                            },
                            "required": [
                                "title",
                                "text"
                            ],
                            "additionalProperties": false
                        }
                    }
                },
                "required": [
                    "segments"
                ],
                "additionalProperties": false
            }
        }
        ### JSON Schema End ###  

        重點指示
            1.	嚴格 JSON
                •	最終輸出必須是單純的 JSON 物件，不能包含多餘的文字、註解或 Markdown 語法。
                •	結構上僅包含 { "segments": [...] } 作為根物件。
                •	segments 裡的每個項目必須包含 "text"和"title"。
                •	"text"：以字串形式放入該段的內容。
            2.	語意解析
                •	當輸入的自然語言或非標準 JSON 中，出現類似「段落」、text：…」等資訊，你需根據上下文，將它們映射到對應的 segments[].text。
                •	若有非法或不認得的屬性、或多餘的文字，請忽略或略過，確保最終符合 Schema。
            3.	多段整合
                •	如果使用者輸入中含有多個段落描述，需將它們合併成 segments 陣列中的多個元素。
                •	每個元素必須同時有 "text"。
            5.	範例
                •	輸入（可能像）：

                    這是一個段落：text = "此處文字123"
                    轉換後輸出：

                    {
                    "segments": [
                        {
                        "title": "這是一個段落",
                        "text": "此處文字123"
                        }
                    ]
                    }

                    輸出格式
                        •	請只輸出一個 JSON 物件，範例如下：

                    {
                    "segments": [
                        {
                        "title": "……",
                        "text": "……",
                        },
                        {
                        "title": "……",
                        "text": "……",
                        }
                    ]
                    }
                •	不要有多餘的文字解釋或其他欄位。"""
        schema = {
            "name": "segments_metadata",
            "strict": True,
            "schema": {
                "type": "object",
                "description": "Schema that describes how transcript segments are structured.",
                "properties": {
                    "segments": {
                        "description": "The segmented results from the transcript. Each item is an individual segment.",
                        "type": "array",
                        "items": {
                            "type": "object",
                            "description": "A single segment containing the text.",
                            "properties": {
                                "title": {
                                    "type": "string",
                                    "description": "The actual title abount the segment."
                                },
                                "text": {
                                    "type": "string",
                                    "description": "The actual text contained within this segment."
                                }
                            },
                            "required": [
                                "title",
                                "text"
                            ],
                            "additionalProperties": False
                        }
                    }
                },
                "required": [
                    "segments"
                ],
                "additionalProperties": False
            }
        }
        messages = [
            {"role": "system", "content": [{"type": "text", "text": prompt}]},
            {
                "role": "user",
                "content": [
                    { "type": "text", "text": """1. 伽利略與物理學的演進
接著我們來看，物理學的演進當中，我們一定會提到這個人——伽利略。在他生存的那個年代，整個學術學派大概分兩種。一種是哲學家學派，主要的代表人物是亞里士多德。這個學派就是憑著看到一個現象之後，憑著自己主觀的思考和純粹的推理來做結論。第二個學派是伽利略所屬的學派，是自然科學家學派，主要代表人物是阿基米德，當然伽利略也是當中的一個很重要的代表人物。在這個學派當中，他們認為應該要依靠科學實踐的方法來做結論。像伽利略，他接受哥白尼的數學觀，認為看到一個現象之後應該去做數學論證，提出假設，而且要設計實驗來檢驗它的原理。在稱謂上，我們會說伽利略是實驗物理學之父。
2. 斜面實驗
他做過什麼事情呢？第一個，我們常見的是斜面實驗。這裡有個斜面，他把小球放手之後往下滑，他要模擬自由落體，只是讓小球在斜面上滑下來。我們知道東西往下掉會越掉越快，那滑行的距離和時間之間有什麼樣的關係？他就在軌道上面放了好多個小鈴鐺，當小球滑過去的時候會撞到鈴鐺發出聲音。如果在等間隔的地方放上鈴鐺，會越滑越快，你聽到的鈴鐺聲會越來越密集。伽利略就來調整這個鈴鐺的間距，讓他聽起來覺得這個聲音一直都是穩定的頻率，比如每隔0.5秒就會叮一次。聽起來每次經過鈴鐺的時間間隔差不多，再從側面去看他們的距離關係。他用這個實驗來驗證數學假設和推論是否正確。
3. 慣性的假想實驗
第二個是慣性的假想實驗。亞里士多德認為，要保持物體的等速度運動，必須對它繼續施加力量。他認為力量是用來保持物體運動的必要條件。但是伽利略不這麼認為。他認為力量不是用來維持物體的運動狀態，而是用來改變物體運動狀態的。他設計了一個假想實驗來說明這一點。如果設計一個光滑的斜面，光滑面實際上不存在，是假想的。如果把球放在光滑的斜面上，它會滑到哪裡去？根據能量守恆，它應該會想要滑回原本的高度，來回滑動。如果把軌道傾斜一點，球放手之後依然會想要回到原本的高度。如果把傾斜的軌道放得很遠，球會一直往右滑動，直到很遠的地方才會爬上斜面。在這個假想實驗中，伽利略認為，如果沒有力量作用，物體應該維持靜止或等速度運動。力量是用來改變運動狀態的。球在水平面上等速度向右移動，如果從側邊推一下，球的方向會轉彎。所以他認為力量是用來改變物體運動狀態的。
4. 單擺實驗
伽利略的另一個實驗是單擺實驗。故事是這樣的：有一天，他去教堂，看到工人在維修吊燈。他發現吊燈在那邊來回擺動，本來大角度擺動，後來越擺越小，但擺動的週期差不多。所以他設計了實驗，用不同的繩長綁著重錘，來回擺動，看看週期和繩長、質量之間的關係。最後他得出結論，單擺擺動的週期等於2π乘上根號（L/G），L是擺繩的長度。他發現單擺在擺動的時候，週期與擺錘質量無關，只要角度不要太大，來回擺動的週期會維持穩定。因為繩長不變，G重力場的值一樣，所以週期幾乎不變。"""}
                ]
            }
        ]

        # 请将以下 URL 替换为实际的 API 端点
        url = 'https://sujfzchcwdlma5-5000.proxy.runpod.net/v1/chat/completions'

        headers = {
            'Content-Type': 'application/json'
            # 如需身份验证，请在此处添加 'Authorization' 字段
        }

        payload = {
            'model': 'Models/Llama-3.3-70B-Instruct',
            'max_tokens': 1024,
            'temperature': 0.6,
            'messages': messages,
            'guided_json':schema
        }

        response = requests.post(url, headers=headers, json=payload)

        if not response.ok:
            raise Exception(f"HTTP error! status: {response.status_code}")

        data = response.json()

        return data

    except Exception as error:
        print('Error:', error)
        raise error

start_time = time.time()
segments = transcribe()
print('Time:', time.time() - start_time)
print(segments)

Time: 76.61299324035645
{'id': 'chatcmpl-b8f34853dce04e2384fa9dc9c77b05ac', 'object': 'chat.completion', 'created': 1739023218, 'model': 'Models/Llama-3.3-70B-Instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'reasoning_content': None, 'content': '{\n    "segments": [\n        {\n            "title": "伽利略與物理學的演進",\n            "text": "物理學的演進當中，我們一定會提到這個人——伽利略。在他生存的那個年代，整個學術學派大概分兩種。一種是哲學家學派，主要的代表人物是亞里士多德。這個學派就是憑著看到一個現象之後，憑著自己主觀的思考和純粹的推理來做結論。第二個學派是伽利略所屬的學派，是自然科學家學派，主要代表人物是阿基米德，當然伽利略也是當中的一個很重要的代表人物。在這個學派當中，他們認為應該要依靠科學實踐的方法來做結論。像伽利略，他接受哥白尼的數學觀，認為看到一個現象之後應該去做數學論證，提出假設，而且要設計實驗來檢驗它的原理。在稱謂上，我們會說伽利略是實驗物理學之父。"\n        },\n        {\n            "title": "斜面實驗",\n            "text": "他做過什麼事情呢？第一個，我們常見的是斜面實驗。這裡有個斜面，他把小球放手之後往下滑，他要模擬自由落體，只是讓小球在斜面上滑下來。我們知道東西往下掉會越掉越快，那滑行的距離和時間之間有什麼樣的關係？他就在軌道上面放了好多個小鈴鐺，當小球滑過去的時候會撞到鈴鐺發出聲音。如果在等間隔的地方放上鈴鐺，會越滑越快，你聽到的鈴鐺聲會越來越密集。伽利略就來調整這個鈴鐺的間距，讓他聽起來覺得這個聲音一直都是穩定的頻率，比如每隔0.5秒就會叮一次。聽起來每次經過鈴鐺的時間間隔差不多，再從側面去看他們的距離關係。他用這個實驗來驗證數學假設和推論是否正確。"\n        },\n        {\n